# Introduction

In [1]:
import pySSV as ssv
import logging
ssv.ssv_logging.set_severity(logging.INFO)

In [2]:
canvas = ssv.canvas()
canvas.dbg_log_context()
canvas.dbg_render_test()

In [4]:
canvas.run(stream_mode="png")

SSVRenderWidget()

In [4]:
canvas.stop()

In [6]:
canvas1 = ssv.canvas()
canvas1.dbg_shader("""
vec4 mainImage( in vec2 fragCoord )
{
    // Normalized pixel coordinates (from 0 to 1)
    vec2 uv = fragCoord/iResolution.xx;
    float aaScale = 1./iResolution.x;

    vec2 mouse = uv-iMouse.xy / iResolution.xx;

    // Time varying pixel color
    vec3 col = vec3(smoothstep(0.9, .95, 1.-length(mouse)));
    col -= 1.-vec3(step(dot(step(abs(mouse), vec2(0.8/iResolution.x, 5./iResolution.x)), vec2(0.5)), 0.5));
    col -= 1.-vec3(step(dot(step(abs(mouse), vec2(5./iResolution.x, 0.8/iResolution.x)), vec2(0.5)), 0.5));

    // Output to screen
    return vec4(vec3(col), 1.0);
}
""")
canvas1.run()

SSVRenderWidget(streaming_mode='jpg')

In [ ]:
canvas2 = ssv.canvas()
canvas2.dbg_shader("""
// Copyright Thomas Mathieson all rights reserved
// https://www.shadertoy.com/view/DsffWM
const float motionBlur = 0.3;
const float aa = 0.6;
const vec3 col1 = vec3(13., 45., 140.)/100.;
const vec3 col2 = vec3(255., 20., 50.)/255.;
const vec3 col3 = vec3(21., 191., 112.)/600.;
const vec3 col4 = vec3(0.35, 1., 0.7)*0.65;
const float speed = 0.1;

float sigmoid(float x)
{
    return 1.*x/(abs(x)+1.);
}
vec3 sigmoid(vec3 x)
{
    return x/(abs(x)+vec3(1.));
}
vec3 saturate(vec3 x)
{
    return clamp(x, 0., 1.);
}
vec3 blend(float x, vec3 c)
{
    c = pow(c, vec3(x+2.));
    return mix(x*c, x*(1.-c), step(x, 0.));
}

float f(vec2 p, float t, vec4 o, vec4 o1, float s, vec4 scale)
{
    vec4 i0 = cos(t+o)*vec4(o.xw, o1.xw);
    vec4 i1 = sin(t+o1)*vec4(o.xw, o1.xw);
    vec4 x0 = i0*s*sin(scale*length(p*o.xy+4.*scale.zw)+o.z+t*o.w);
    vec4 x1 = i1*s*sin(scale*length(p*o1.xy)+o1.z+t*o1.w);
    return sigmoid(dot(x0+x1, vec4(1.)));
}

vec3 scene(float t, float emphasis, vec2 uv)
{
    // "Beautiful" randomness, tuned for aesthetics, not performance
    vec2 p = uv * 3.;
    t += 160.;
    t *= speed;
    vec4 scale = vec4(sin(t*vec3(0.25, .5, .75)), cos(t*.95))*.25+.5;
    float s0 = f(p, t, vec4(6.,9.,2.,1.5), vec4(2.,9.,7.,3.), .25, scale);
    float s1 = f(p, t, vec4(2.,6.5,1.5,4.0), vec4(3.,2.5,3.8,1.6), .5, scale);
    float s2 = sigmoid(s0/s1)*0.5;
    float s3 = f(p, t, vec4(2.,9.,7.,3.), vec4(6.,3.,2.,1.5), .125, scale);
    float s6 = f(p*1.5, t, vec4(6.,4.,8.,2.5), vec4(3.2,1.6,9.7,7.9), .25, scale);
    float s7 = f(p*1.3, t, vec4(2.,6.5,1.5,4.0), vec4(3.,2.5,3.8,1.6), .5, scale);
    float s8 = sigmoid(s6/s7+s0)*0.7;
    
    vec3 c = vec3(sigmoid((blend(s8,col1)+blend(s2,col2)+blend(s1,col3)+s7*1.)*1.1)*.7+.5);
    float grad = sigmoid(pow(length(uv*2.-1.)+s3*.3, 5.))*1.5;
    float accent = 1.-sigmoid((pow(2.5, abs(sigmoid(s8+s0+s1))-1.)-.45-(emphasis*0.1))*1000./(1.+30.*grad+20.*emphasis));
    c = mix(c, c.r*.3+col4*.8, accent);
    return clamp(vec3(c), 0., 1.);
}

vec4 mainImage( in vec2 fragCoord )
{
    // Normalized pixel coordinates (from 0 to 1)
    vec2 uv = fragCoord/iResolution.xx;
    float aaScale = 1./iResolution.x;

    vec2 mouse = uv-iMouse.xy /iResolution.xx;
    float emp = sigmoid(1./pow(length(mouse*1.), 1.8)*.02);

    // Time varying pixel color
    vec3 col = scene(iTime, emp, uv);
    //col     += scene(iTime + motionBlur*0.001, emp, uv + aaScale*aa*vec2(0.,1.))
    //         + scene(iTime + motionBlur*0.002, emp, uv + aaScale*aa*vec2(1.,0.));
    //col /= 3.;

    // Output to screen
    return vec4(vec3(col), 1.0);
}
""")
canvas2.run()

In [ ]:
canvas.stop()
canvas1.stop()
canvas2.stop()